# Analiza danych z ankiety stackoverflow 2020
## Kamil Misiak

W tym sprawozdaniu przeprowadzę analizę zbioru danych, zawierającego dane z ankiety przeprowadzonej przez portal stackoverflow. Zbiór danych zostanie załadowany, oczyszczony i przeanalizowany.

## Agenda
1. Załadowanie zbioru danych i wstępne statystyki
2. Przygotowanie danych
3. Analiza i wizualizacja danych
4. Pomiar dokładności klasyfikacji
5. Podsumowanie

## Krok 1: Załadowanie zbioru danych i wstępne statystyki

In [ ]:
# Przygotowanie danych
import pandas as pd
import numpy as np

# Wizualizacja
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Dostosowanie wyglądu wizualizacji
sns.set_style('darkgrid')
plt.rcParams['font.size'] = 14
plt.rcParams['figure.figsize'] = (9, 5)
plt.rcParams['figure.facecolor'] = '#00000000'

# Klasyfikacja
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

# Scieżki do plików (dane, schemat)
data_file_name = './data/survey_results_public.csv'
schema_file_name = './data/survey_results_schema.csv'

# Załadowanie zbioru
data_raw = pd.read_csv(data_file_name, sep=',', header=0)

#### Następnie przyjrzmy się atrybutom które występują w zbiorze danych

In [ ]:
data_raw.columns

#### Następnie wyświetlimy 5 pierwszych rekordów

In [ ]:
data_raw.head()

#### Oraz końcowe 5 rekordów

In [ ]:
data_raw.tail()

#### Podgląd zbioru

In [ ]:
print(data_raw.shape)
print('\n')
print(data_raw.dtypes)

#### Liczba atrybutów oraz typ danych

In [ ]:
data_raw.info()

Jak widać w zbiorze mamy 64461 rekordy, które zawierają 61 atrybutów.
#### Następnie wyświetlimy podstawowe statystki opisowe atrybutów

In [ ]:
data_raw.describe()

#### Do zbioru danych dodatkowo dołączony jest plik zawierający opisy zawartości atrybutów

In [ ]:
schema_raw = pd.read_csv(schema_file_name, index_col='Column')
pd.set_option('display.max_rows', schema_raw.shape[0]+1)
pd.set_option('max_colwidth', None)
schema_raw

Następnie przygotujemy zmienną, dzięki której będziemy mogli odczytywać pytanie należące do danej kolumny.

In [ ]:
schema_raw = pd.read_csv(schema_file_name, index_col='Column').QuestionText

In [ ]:
# Dowolna nazwa kolumny
schema_raw['WorkWeekHrs']

## Krok 2: Przygotowanie danych

Chociaż odpowiedzi na ankietę zawierają bogactwo informacji, moja analiza zostanie ograniczona do następujących obszarów:

1. Dane demograficzne respondentów badania,
2. Dystrybucja umiejętności programistycznych, doświadczenia i preferencji,
3. Informacje, preferencje i opinie dotyczące zatrudnienia.

Wybieramy podzbiór atrubutów z odpowiednimi danymi do naszej analizy.

In [ ]:
selected_columns = [
    # Dane demograficzne
    'Country',
    'Age',
    'Gender',
    'EdLevel',
    'UndergradMajor',
    # Doświadczenie programistyczne
    'Hobbyist',
    'Age1stCode',
    'YearsCode',
    'YearsCodePro',
    'LanguageWorkedWith',
    'LanguageDesireNextYear',
    'NEWLearn',
    'NEWStuck',
    # Zatrudnienie
    'Employment',
    'DevType',
    'WorkWeekHrs',
    'JobSat',
    'JobFactors',
    'NEWOvertime',
    'NEWEdImpt'
]

In [ ]:
len(selected_columns)

Skopiujemy dane zawierającą powyższe kolumny, do nowej ramki danych o nazwie "data". Aby kontynuować dalsze przygotowanie danych bez wpływu na orginalny zbiór danych

In [ ]:
data = data_raw[selected_columns].copy(deep=True)

In [ ]:
schema = schema_raw[selected_columns]

Wyświetlimy podstawowe informacje o ramce danych

In [ ]:
data.shape

In [ ]:
data.info()

Następnie określimy pradwidłowe typy dla naszy danych

Zaczniemy od atrybutu "Age1stCode"

In [ ]:
schema['Age1stCode']

Według pytania, atrybut powinnien zawierać wartości numeryczne określające wiek osoby ankietowanej

In [ ]:
data['Age1stCode'].unique()

Jak widać trafiają się również ciągi znaków np, "Older than 85". Zamiennimy te wartości na wartości puste, ponieważ określają one wartości odstające, które nie są przydatne w anzalizie. Wykonamy tą samą operacje dla 'YearsCode' oraz 'YearsCodePro' ponieważ te atrybuty zawierają podobne wartości. Poniższe operacje przekształcają wartości liczobwe na liczby, a ciągi znaków na wartości puste.

In [ ]:
data['Age1stCode'] = pd.to_numeric(data['Age1stCode'], errors='coerce')
data['YearsCode'] = pd.to_numeric(data['YearsCode'], errors='coerce')
data['YearsCodePro'] = pd.to_numeric(data['YearsCodePro'], errors='coerce')

Wyświetlimy podstawe informacje po przekształceniu wartości

In [ ]:
data.describe()

Wygląda na to, że występuje problem z kolumną wieku, ponieważ minimalna wartość to 1, a maksymalna to 279. Jest to częsty problem z ankietami, odpowiedzi mogą zawierać nieprawidłowe wartości z powodu przypadkowych lub celowych błędów podczas odpowiadania. Prostym rozwiązanie będzie usunięcie wierszy, w których wiek jest wyższy niż 100 lat lub niższy niż 10 lat. Możemy to zrobić za pomocą metody .drop

In [ ]:
data.drop(data[data['Age'] < 10].index, inplace=True)
data.drop(data[data['Age'] > 100].index, inplace=True)

To samo dotyczy WorkWeekHrs. Usuniemy wpisy, w których wartość kolumny jest większa niż 140 godzin. (około 20 godzin dziennie).

In [ ]:
data.drop(data[data['WorkWeekHrs'] > 140].index, inplace=True)

Kolumna płeć pozwala również na wybranie wielu opcji. Usuniemy wartości zawierające więcej niż jedną opcję, aby uprościć naszą analizę.

In [ ]:
data['Gender'].value_counts()

In [ ]:
data.where(~(data['Gender'].str.contains(';', na=False)), np.nan, inplace=True)

Wyczyściliśmy teraz i przygotowaliśmy zbiór danych do analizy. Przyjrzyjmy się próbce wierszy.

In [ ]:
data.sample(10)

## Krok 3: Analiza i wizualizacja danych

W pierwszym kroku przyjrzymy się danym demograficznych respondentów, tj. Kraju, wieku, płci, poziomu wykształcenia, poziomu zatrudnienia itp. Konieczne jest zbadanie tych zmiennych, aby zrozumieć, jak reprezentatywna jest ankieta dla światowej społeczność programistów.

#### Atrybut "Country"

Przyjrzyjmy się liczbie krajów, z których pochodzą odpowiedzi w ankiecie i wytypujmy dziesięć krajów z największą liczbą odpowiedzi.

In [ ]:
schema['Country']

In [ ]:
data['Country'].nunique()

Kraje o największej liczbie respondentów możemy zidentyfikować metodą value_counts.

In [ ]:
top_countries = data['Country'].value_counts().head(10)
top_countries

In [ ]:
plt.figure(figsize=(12,6))
plt.xticks(rotation=75)
plt.title(schema.Country)

sns.barplot(x=top_countries.index, y=top_countries);

Oraz przedstawimy dane na mapie świata z pomocą elementu "dash".

In [ ]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import pycountry
import pycountry_convert as pc

# Załadowanie danych
top_countries_df = data['Country'].value_counts().rename_axis('country').reset_index(name='counts')

countries_alpha_3 = {}
countries_alpha_2 = {}
for country in pycountry.countries:
    countries_alpha_3[country.name] = country.alpha_3
    countries_alpha_2[country.name] = country.alpha_2
        
top_countries_df['iso_alpha_3'] = [countries_alpha_3.get(country, 'Unknown code') for country in top_countries_df['country']]
top_countries_df['iso_alpha_2'] = [countries_alpha_2.get(country, 'Unknown code') for country in top_countries_df['country']]

for index, row in top_countries_df.iterrows():
    if row['iso_alpha_2'] != 'Unknown code' and row['iso_alpha_2'] != 'TL':
        top_countries_df.at[index, 'continent'] = pc.country_alpha2_to_continent_code(row['iso_alpha_2'])
    else:
        top_countries_df.at[index, 'continent'] = 'Other'
    
fig = px.scatter_geo(top_countries_df, locations="iso_alpha_3", color="continent",
                     hover_name="country", size="counts",
                     projection="natural earth")

# Uruchomienie widgetu
app = JupyterDash(__name__)
app.layout = html.Div([
    html.P("Kraje:"),
    dcc.Graph(figure=fig),
])


app.run_server(mode='inline', use_reloader=False)

Wygląda na to, że nieproporcjonalnie duża liczba respondentów pochodzi z USA i Indii, prawdopodobnie dlatego, że ankieta jest prowadzona w języku angielskim, a kraje te mają największą populację anglojęzyczną. Można zakładać, że ankieta może nie być reprezentatywna dla globalnej społeczności programistów - zwłaszcza z krajów nieanglojęzycznych.

#### Atrybut "Age"
Rozkład wieku respondentów jest kolejnym czynnikiem, na który warto zwrócić uwagę. Do wizualizacji użyjemy histogramu.

In [ ]:
plt.figure(figsize=(12, 6))
plt.title(schema['Age'])
plt.xlabel('Wiek')
plt.ylabel('Liczba respondentów')

plt.hist(data['Age'], bins=np.arange(10,80,5), color='purple');

Wydaje się, że duży odsetek respondentów ma od 20 do 45 lat. Jest to w pewnym stopniu reprezentatywne dla społeczności programistów. Wielu młodych ludzi podjęło się nauki informatyki jako kierunku studiów lub zawodu w ciągu ostatnich 20 lat.

#### Atrybut "Gender"
Przyjrzyjmy się rozkładowi odpowiedzi dla płci. Powszechnie wiadomo, że kobiety są niedostatecznie reprezentowane w społeczności programistów.

In [ ]:
schema['Gender']

In [ ]:
gender_counts = data['Gender'].value_counts()
gender_counts

In [ ]:
plt.figure(figsize=(12,6))
plt.title(schema.Gender)
plt.pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', startangle=180);

Tylko około 8% respondentów, którzy odpowiedzieli na pytanie, identyfikuje się jako kobiety lub osoby niebinarne. Liczba ta jest niższa niż ogólny odsetek kobiet i niebinarnych płci w społeczności programistów (szacowany na około 12%).

#### Atrybutu "Education Level"
Formalne wykształcenie informatyczne jest często uważane za niezbędny warunek zostania programistą. Istnieje jednak wiele bezpłatnych kursów i samouczków dostępnych w internecie, aby nauczyć się programowania. Porównamy poziom wykształcenia respondentów, aby uzyskać wgląd. Użyjemy do tego celu poziomego wykresu słupkowego.

In [ ]:
sns.countplot(y=data['EdLevel'])
plt.xticks(rotation=75);
plt.title(schema['EdLevel'])
plt.ylabel(None);

Wydaje się, że ponad połowa respondentów posiada tytuł licencjata lub magistra, więc większość programistów wydaje się mieć wykształcenie wyższe. Jednak z samego tego wykresu nie wynika jasno, czy posiadają dyplom z informatyki.

Następnie przyjrzymy się głównemu kierunkowi nauki, jaki wybrali respondenci.

In [ ]:
schema.UndergradMajor

In [ ]:
undergrad_pct = data['UndergradMajor'].value_counts() * 100 / data['UndergradMajor'].count()

sns.barplot(x=undergrad_pct, y=undergrad_pct.index)

plt.title(schema['UndergradMajor'])
plt.ylabel(None);
plt.xlabel('Procent');

Okazuje się, że 40% programistów z wyższym wykształceniem ma kierunek inny niż informatyka - co jest bardzo zachęcające. Wydaje się to sugerować, że chociaż wykształcenie wyższe jest ogólnie pomocne, nie jest konieczne studiowanie informatyki, aby odnieść sukces jako programista.

#### Atrybutu "Employment"
Praca na zlecenie lub własna działalność gospodarcza to częsty wybór wśród programistów, więc byłoby interesujące porównać podział na pracę w pełnym wymiarze godzin, w niepełnym wymiarze godzin i pracę na własny rachunek. Zwizualizujmy dane z kolumny Zatrudnienie.

In [ ]:
schema['Employment']

In [ ]:
(data['Employment'].value_counts(normalize=True, ascending=True)*100).plot(kind='barh', color='g')
plt.title(schema['Employment'])
plt.xlabel('Procent');

Wygląda na to, że blisko 10% respondentów jest zatrudnionych w niepełnym wymiarze godzin lub jako freelancerzy.

Pole DevType zawiera informacje o rolach pełnionych przez respondentów. Ponieważ pytanie pozwala na wiele odpowiedzi, kolumna zawiera listy wartości oddzielone średnikiem, co utrudnia analizę.

In [ ]:
schema['DevType']

In [ ]:
data['DevType'].value_counts()

Zdefiniujmy funkcję pomocniczą, która zamienia kolumnę zawierającą listy wartości (np. data['DevType']) w ramkę danych z jedną kolumną dla każdej możliwej opcji.

In [ ]:
def split_multicolumn(col_series):
    result_df = col_series.to_frame()
    options = []
    # Pętla po wszystkich atrybutach
    for idx, value  in col_series[col_series.notnull()].iteritems():
        # Podział wartośći według średnika
        for option in value.split(';'):
            # Jężeli nie występuje taka wartość, dodanie nowej
            if not option in result_df.columns:
                options.append(option)
                result_df[option] = False
            # Oznaczenie wartośći jak True
            result_df.at[idx, option] = True
    return result_df[options]

In [ ]:
dev_type_df = split_multicolumn(data['DevType'])

In [ ]:
dev_type_df

Zmienna "dev_type_df" ma jedną kolumnę dla każdej opcji, którą można wybrać jako odpowiedź. Jeśli respondent wybrał opcję wtedy kolumna przyjmuje wartość True, W przeciwnym razie wartość False.

Możemy teraz użyć sum według kolumn, aby zidentyfikować najpowszechniejsze role.

In [ ]:
dev_type_totals = dev_type_df.sum().sort_values(ascending=False)
dev_type_totals

Jak widać najwięcej programistów zajmuję się backendem oraz frontendem, co nie jest zaskakujące ponieważ świat IT obecnie w głównej mierze skupia się na technologiach webowych.

#### Najpopularniejszy język programowania w roku 2020?

Aby odpowiedzieć, możemy użyć kolumny LanguageWorkedWith. Podobnie jak w przypadku atrybutu "DevType", respondenci mogli wybrać tutaj wiele opcji.

In [ ]:
data['LanguageWorkedWith']

Najpierw podzielimy tę kolumnę na ramkę danych zawierającą kolumnę każdego języka wymienionego w opcjach (tak samo jak w przypadku kolumny "DevType").

In [ ]:
languages_worked_df = split_multicolumn(data['LanguageWorkedWith'])

In [ ]:
languages_worked_df

Wśród opcji uwzględniono łącznie 25 języków. Zsumujmy je, aby określić procent respondentów, którzy wybrali każdy język.

In [ ]:
languages_worked_percentages = languages_worked_df.mean().sort_values(ascending=False) * 100
languages_worked_percentages

Możemy zwizualizować te informacje za pomocą poziomego wykresu słupkowego.

In [ ]:
plt.figure(figsize=(12, 12))
sns.barplot(x=languages_worked_percentages, y=languages_worked_percentages.index)
plt.title("Języki używane w ostatnich latach");
plt.xlabel('liczba');

Nic dziwnego, że Javascript i HTML / CSS znajdują się na szczycie, ponieważ tworzenie stron internetowych jest obecnie jedną z najbardziej poszukiwanych umiejętności. Jest to również jeden z najłatwiejszych do rozpoczęcia. SQL jest niezbędny do pracy z relacyjnymi bazami danych, nic więc dziwnego, że większość programistów regularnie korzysta z SQL. Wydaje się, że Python jest popularnym wyborem dla innych form programowania, wyprzedzając Javę, która była branżowym standardem w tworzeniu serwerów i aplikacji przez ponad dwie dekady.

#### Jak ważne jest by budować karierę w młodym wieku?

Stwórzmy wykres punktowy przedstawiający zależność "Age" od "YearsCodePro" (tj. Wiek i Lata doświadczenia w kodowaniu), aby odpowiedzieć na to pytanie.

In [ ]:
schema['YearsCodePro']

In [ ]:
sns.scatterplot(x='Age', y='YearsCodePro', hue='Hobbyist', data=data)
plt.xlabel("Wiek")
plt.ylabel("Lata doświadczenia w programowaniu");

Punkty widoczne są na całym wykresie, co może sugerować, że możesz zacząć programować zawodowo w każdym wieku. Wiele osób, które zajmują się programowaniem zawodowo od kilkudziesięciu lat, również lubi to robić jako hobby.

Możemy również zobaczyć rozkład kolumny "Age1stCode", aby zobaczyć, kiedy respondenci próbowali programować po raz pierwszy

In [ ]:
plt.title(schema.Age1stCode)
sns.histplot(x=data['Age1stCode'], bins=30, kde=True);

Jak można się było spodziewać, większość ludzi miała styczność z programowaniem przed czterdziestym rokiem życia. Jednak są ludzie w każdym wieku i ze wszystkich środowisk uczących się kodowania.

## Krok 4: Pomiar dokładności klasyfikacji

W ostatnim kroku spróbujemy sklasyfikować na podstwie pozyskanych danych, z jakiego kraju pochodzi respondent.

Użyjemy do tego zadania atrybutów: Country, Age, YearsCodePro, Gender, EdLevel, Employment

In [ ]:
selected_columns = [
    'Country',
    'Age',
    'YearsCodePro',
    'Gender',
    'EdLevel',
    'Employment',
]

X_data = data[selected_columns].copy(deep=True)
X_data

Przygotujemy zmienną le, służacą do kodowania wartości atrybutów

In [ ]:
le = preprocessing.LabelEncoder()

Aby uruchomić algorytm klasyfikacji musimy usunąć brakujące wartośći z atrybutu Age. Zrobimy to tym samym sposobem co w sprawozdaniu titanic.

In [ ]:
mean = X_data["Age"].mean()
std = X_data["Age"].std()
is_null = X_data["Age"].isnull().sum()

# Losujemy wartości z przedziału wartości średniej i odchylenia standardowego atrybutu "Age"
rand_age = np.random.randint(mean - std, mean + std, size = is_null)

# Uzpełniamy puste wartości w atrybucie "Age" wylosowanymi wartościami
age_slice = X_data["Age"].copy()
age_slice[np.isnan(age_slice)] = rand_age
X_data["Age"] = age_slice
X_data["Age"] = X_data["Age"].astype(int)
X_data["Age"].isnull().sum()
X_data["Age"] = pd.qcut(X_data['Age'], 3)
le.fit(X_data["Age"])
X_data["Age"] = le.transform(X_data["Age"])

Podobnie postąpimy z atrybutem YearsCodePro

In [ ]:
mean = X_data["YearsCodePro"].mean()
std = X_data["YearsCodePro"].std()
is_null = X_data["YearsCodePro"].isnull().sum()

# Losujemy wartości z przedziału wartości średniej i odchylenia standardowego atrybutu "Age"
rand_age = np.random.randint(mean - std, mean + std, size = is_null)

# Uzpełniamy puste wartości w atrybucie "Age" wylosowanymi wartościami
age_slice = X_data["YearsCodePro"].copy()
age_slice[np.isnan(age_slice)] = rand_age
X_data["YearsCodePro"] = age_slice
X_data["YearsCodePro"] = X_data["YearsCodePro"].astype(int)
X_data["YearsCodePro"].isnull().sum()
X_data["YearsCodePro"] = pd.qcut(X_data['YearsCodePro'], 3)
le.fit(X_data["YearsCodePro"])
X_data["YearsCodePro"] = le.transform(X_data["YearsCodePro"])

In [ ]:
X_data = X_data.dropna(axis=0, how='any')

In [ ]:
total = X_data.isnull().sum().sort_values(ascending=False)
percent_1 = X_data.isnull().sum()/X_data.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(20)

Jak widać udało się całkowicie pozbyć brakujących wartości.

In [ ]:
# Klasa pomocnicza do operacji na danych
class ChainedAssignent:
    def __init__(self, chained=None):
        acceptable = [None, 'warn', 'raise']
        assert chained in acceptable, "chained must be in " + str(acceptable)
        self.swcw = chained

    def __enter__(self):
        self.saved_swcw = pd.options.mode.chained_assignment
        pd.options.mode.chained_assignment = self.swcw
        return self

    def __exit__(self, *args):
        pd.options.mode.chained_assignment = self.saved_swcw

Mapujemy wartości atrybutu Gender

In [ ]:
le.fit(X_data['Gender'])
with ChainedAssignent():
    X_data['Gender'] = le.transform(X_data['Gender'])

Mapujemy wartości atrybutu EdLevel

In [ ]:
le.fit(X_data['EdLevel'])
with ChainedAssignent():
    X_data['EdLevel'] = le.transform(X_data['EdLevel'])

Mapujemy wartości atrybutu Employment

In [ ]:
le.fit(X_data['Employment'])
with ChainedAssignent():
    X_data['Employment'] = le.transform(X_data['Employment'])

Przegląd danych w zbiorze do klasyfikacji.

In [ ]:
X_data.info()

Aby ułatwić zadanie algorytmowi klasyfikacji, zostawimy tylko 19 najbardziej popularnych krajów, a reszte zaliczymy do grupy "Other"

In [ ]:
freq = X_data['Country'].value_counts()
with ChainedAssignent():
    X_data['Country'][~X_data['Country'].isin(freq.index[:19])] = 'Other'
    X_data['Country'] = X_data['Country'].astype('category')
X_data['Country'].unique()

In [ ]:
top_countries = X_data['Country'].value_counts().head(20)
top_countries
plt.figure(figsize=(12,6))
plt.xticks(rotation=75)
plt.title(schema.Country)

sns.barplot(x=top_countries.index, y=top_countries);

Jak widać na wykresie, najbardziej popularne grupy to Stany zjednoczone, Indie, i inne

Następnie przygotujemy dane dla algorytmu klasyfikacji

In [ ]:
Y_data = X_data['Country'].copy();
X_data = X_data.drop('Country', axis=1)

Kodujemy wartośći atrybutu Country

In [ ]:
le.fit(Y_data)
Y_data = le.transform(Y_data)

I wywołamy algorytm klasyfikacji

In [ ]:
decision_tree = DecisionTreeClassifier()
scores = cross_val_score(decision_tree, X_data, Y_data, cv=10)
scores
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Udało się uzyskać dokładność na poziomie około 29%.

## Krok 5: Podsumowanie

Z ankiety pożna wyciągnąć wiele wniosków. Wykonana tutaj analiza stanowi tylko małą część tego, czego można się dowiedzieć z analizowanej ankiety. Oto podsumowanie kilku z znalezionych inforamcji:

Na podstawie danych demograficznych respondentów badania możemy wywnioskować, że badanie jest w pewnym stopniu reprezentatywne dla całej społeczności programistów. Jednak ma mniej odpowiedzi od programistów z krajów nieanglojęzycznych oraz kobiet i niebinarnych płci.

Społeczność programistów nie jest tak różnorodna, jak mogłoby się wydawać. Chociaż sytuacja się poprawia, powinniśmy dołożyć większych starań, aby wspierać i zachęcać niedostatecznie reprezentowane społeczności, czy to pod względem wieku, kraju, rasy, płci, czy w inny sposób.

Chociaż większość programistów ma wyższe wykształcenie, dość duży odsetek nie skończyła informatyki jako głównego kierunku studiów. Dlatego dyplom z informatyki nie jest obowiązkowy do nauki kodowania lub budowania kariery programistycznej.

Znaczny odsetek programistów pracuje w niepełnym wymiarze godzin lub jako freelancerzy, co może być świetnym sposobem na wejście w tę dziedzinę, zwłaszcza gdy dopiero zaczynasz.

Javascript i HTML / CSS to najczęściej używane języki programowania w 2020 roku, tuż za nimi plasują się SQL i Python.

Wydaje się, że programiści na całym świecie pracują średnio około 40 godzin tygodniowo, z niewielkimi różnicami w zależności od kraju.

Można się uczyć i zacząć programować zawodowo w każdym wieku.